<a href="https://colab.research.google.com/github/eyupdalan/BLM6114-hw2/blob/main/BLM6114_hw2.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

# Hesaplamalı Anlambilim Dersi 2.Ödevi
Konusu: Sınıflandırma

## Package installations

In [ ]:
!pip install datasets torch scikit-learn transformers # uncomment if necessary

### Necessary imports

In [ ]:
import torch
import random
import re

from datasets import load_dataset
from sklearn.metrics import accuracy_score
from transformers import AutoTokenizer, AutoModelForCausalLM, pipeline

## Dataset preperations
https://huggingface.co/datasets/turkish-nlp-suite/TrGLUE

sst2 için eğitim kümesini 60K-->5K, test kümesini 9K-->1K düşürerek kullanınız.

> TrSST-2 The Stanford Sentiment Treebank is a sentiment analysis dataset includes sentences from movie reviews, annotated by human annotators. The task is to predict the sentiment of a given sentence. Our dataset is compiled from movie review websites BeyazPerde.com and Sinefil.com, both reviews and sentiment ratings are compiled from those websites. Here we offer a binary classification task to be compatible with the original GLUE task, however we offer a 10-way classification challenge in this dataset's standalone HuggingFace repo.

cola için eğitim kümesini 8K-->5K, test kümesini değiştirmeden kullanınız.

> TrCOLA The original Corpus of Linguistic Acceptability consists of sentences compiled from English literature textbooks. The task is to determine if the sentences are grammatically correct and acceptable sentences. Our corpus is also compiled from Turkish linguistic textbooks and include morphological, syntactic and semantic violations. This dataset also has a standalone repo on HuggingFace.


In [ ]:
# SST2
sst2 = load_dataset("turkish-nlp-suite/TrGLUE", "sst2")
sst2_train = sst2['train'].shuffle(seed=42).select(range(5000))  # Eğitim: 60K -> 5K
sst2_test = sst2['test'].shuffle(seed=42).select(range(1000))  # Test: 9K -> 1K

# CoLA
cola = load_dataset("turkish-nlp-suite/TrGLUE", "cola")
cola_train = cola['train'].shuffle(seed=42).select(range(5000))  # Eğitim: 5K
cola_test = cola['test']  # Test: Tüm test kümesi



/usr/local/lib/python3.11/dist-packages/huggingface_hub/utils/_auth.py:94: UserWarning: 
The secret `HF_TOKEN` does not exist in your Colab secrets.
To authenticate with the Hugging Face Hub, create a token in your settings tab (https://huggingface.co/settings/tokens), set it as secret in your Google Colab and restart your session.
You will be able to reuse this secret in all of your notebooks.
Please note that authentication is recommended but still optional to access public models or datasets.
  warnings.warn(


## Check imported data

In [ ]:
sst_train_len = len(sst2_train)
sst_test_len = len(sst2_test)
cola_train_len = len(cola_train)
cola_test_len = len(cola_test)

print(f"SST2 eğitim kümesi uzunluğu: {sst_train_len}")
print(f"SST2 test kümesi uzunluğu: {sst_test_len}")
print(f"CoLA eğitim kümesi uzunluğu: {cola_train_len}")
print(f"CoLA test kümesi uzunluğu: {cola_test_len}")

# sst2_train verisinin ilk 10 satırını yazdır
print("sst2_train:")
for i in range(10):
    print(sst2_train[i])

# sst2_test verisinin ilk 10 satırını yazdır
print("\nsst2_test:")
for i in range(10):
    print(sst2_test[i])

# cola_train verisinin ilk 10 satırını yazdır
print("\ncola_train:")
for i in range(10):
    print(cola_train[i])

# cola_test verisinin ilk 10 satırını yazdır
print("\ncola_test:")
for i in range(10):
    print(cola_test[i])

SST2 eğitim kümesi uzunluğu: 5000
SST2 test kümesi uzunluğu: 1000
CoLA eğitim kümesi uzunluğu: 5000
CoLA test kümesi uzunluğu: 1000
sst2_train:
{'sentence': 'film başladığında bir müddet izlemesem mi acaba, sıkıcı bir filme benziyor diye düşündüm. Ancak ilerledikçe iyi ki devam etmişim dedim. Farklı ve gerçekten iyi bir film. İyi seyirler..', 'label': 1}
{'sentence': 'Film gerçek hayata göndermeler yapıyor. Filmi çok başarılı buldum gerçekten ender güzel yapımlardan. Her sahnesi özenle hazırlanmış. Oyunculuklar, replikler harika, fazla söze gerek yok mutlaka izlenmesi gereken bir yapım, sinema severlere duyrulur :)', 'label': 1}
{'sentence': 'harika bir animasyon. çok eğlenceli. tıpkı birincisi gibi.', 'label': 1}
{'sentence': "Ümit ünal filmlerini genelde beğenirim bu filmde keyifle izlediğim filmlerden hele ki evdeki ilk ayaküstü sorgulama sahnesi çok bizden kara mizah ama gerilimi daha çok mizahı daha az olsaydı diye de ummadım değil... Demet Evgar'ın katmanlı karakterini izlemek he

## Model installments

1. https://huggingface.co/ytu-ce-cosmos/Turkish-Llama-8b-DPO-v0.1
2. https://huggingface.co/google/gemma-2-9b-it
3. https://huggingface.co/Trendyol/Llama-3-Trendyol-LLM-8b-chat-v2.0
4. https://huggingface.co/TURKCELL/Turkcell-LLM-7b-v1
5. https://huggingface.co/WiroAI/wiroai-turkish-llm-9b

In [ ]:
model_names = [
    "meta-llama/Meta-Llama-3.1-8B-Instruct",
    "ytu-ce-cosmos/Turkish-Llama-8b-DPO-v0.1",
    "google/gemma-2-9b-it",
    "Trendyol/Trendyol-LLM-8b-chat-v2.0",
    "TURKCELL/Turkcell-LLM-7b-v1",
    "WiroAI/wiroai-turkish-llm-9b",
]
def generate_llm (model_name):
    tokenizer = AutoTokenizer.from_pretrained(model_name)
    model = AutoModelForCausalLM.from_pretrained(
        model_name,
        device_map="auto",
        torch_dtype=torch.float16,
        pad_token_id=tokenizer.eos_token_id
    )
    llm = pipeline("text-generation", model=model, tokenizer=tokenizer)
    return llm

# Modelleri yükleme
models = {
    "cosmos_dpo": generate_llm("ytu-ce-cosmos/Turkish-Llama-8b-DPO-v0.1"),
    #"gemma2_9b": generate_llm("google/gemma-2-9b-it"),
    #"trendyol_llm": generate_llm("Trendyol/Trendyol-LLM-8b-chat-v2.0"),
    #"turkcell_llm":generate_llm("TURKCELL/Turkcell-LLM-7b-v1"),
    #"wiroai_llm": generate_llm("WiroAI/wiroai-turkish-llm-9b"),
}


Loading checkpoint shards:   0%|          | 0/4 [00:00<?, ?it/s]

Device set to use cuda:0


In [40]:
def build_prompt(sentence):
  return f'Cümle: "{sentence}"\nBu cümledeki duygu olumlu mu? Lütfen sadece "Evet" veya "Hayır" şeklinde cevap verin.\n'

def build_prompts_with_examples(sentence, example_data, shot_count):
  prompt = build_prompt(sentence)
  selected_examples = example_data.select(range(shot_count))
  examples = ""
  for i in range(shot_count):
    cevap = ""
    if(selected_examples["label"][i] == 1):
      cevap = "Evet"
    else:
      cevap = "Hayır"
    examples += f'Cümle: "{selected_examples["sentence"][i]}"\nCevap: {cevap}\n'

  return f"{examples}\nYukarıdaki cümleleri değerlendirin\n{prompt}"


def run_prompt(prompt, model_name="cosmos_dpo", max_new_tokens=150):
  output = models[model_name](prompt, max_new_tokens=max_new_tokens, do_sample=False)[0]['generated_text']
  response = output[len(prompt):].strip()
  return response

def run_bulk_prompt(prompts, model_name="cosmos_dpo", max_new_tokens=150):
  outputs = models[model_name](prompts, max_new_tokens=max_new_tokens, do_sample=False)
  generated_outputs = []
  for output in outputs:
    generated = output[0]['generated_text'][len(prompts[outputs.index(output)]):].lower().strip()  # İstem uzunluğunu çıkar
    generated_outputs.append(generated)

  return generated_outputs

def predict(sentence):
    prompt = build_prompt(sentence)
    output = run_prompt(prompt)
    generated = output.lower()
    if "evet" in generated:
        return 1
    elif "hayır" in generated:
        return 0
    else:
        return -1  # belirsiz durum

def bulk_predict(sentences):
  prompts = [build_prompt(sentence) for sentence in sentences]
  outputs = run_bulk_prompt(prompts)
  pred_labels = []
  for i, output in enumerate(outputs):
    if "evet" in output:
      pred_labels.append(1)
    elif "hayır" in output:
      pred_labels.append(0)
    else:
      pred_labels.append(-1)
  return pred_labels

def bulk_predict_for_few_shot(sentences, example_data, shot_count):
  prompts = [build_prompts_with_examples(sentence, example_data, shot_count) for sentence in sentences]
  outputs = run_bulk_prompt(prompts)
  pred_labels = []
  for i, output in enumerate(outputs):
    if "evet" in output:
      pred_labels.append(1)
    elif "hayır" in output:
      pred_labels.append(0)
    else:
      pred_labels.append(-1)
  return pred_labels

## Prompt check
#prompt = """
#Cümle: "Benim kalem masa."
#Bu cümle dil bilgisi açısından doğru mu? (Evet / Hayır)
#"""

#print(run_prompt(prompt))

In [41]:
print(sst2_test["sentence"][:5])
print(sst2_test["label"][:5])

print(sst2_test.select(range(3))["sentence"])

['Sene olmuş 2013, hala iki yaprak kıpırtısını korku filmi diye yutturmaya çalışanlar var. Efekt (ki film boyunca 1 veya 2 kez göreceksiniz) vasatın altında. Hikaye işe yaramaz, kurgu diye bir şey yok. E ne yapıyorsunuz, film çekiyoruz. Oldu!', 'İnsanın başına hiç beklemediği şeyler gelebilir gibi kısa ve klişe bir temayı filme biraz olsun anlam katmak için mi yaptılar ne...', "Etkileyici bir kişilik Desmond Doss. Kesinlikle tarihe geçen bir hikayesi var. Mel Gibson bu gibi kahramanlık hikayelerini her zaman sevmiştir. Hacksaw Ridge&te de iyi iş çıkardığını söyleyebiliriz. Aslında izlerken pek çok mantık hatası var gibi gelse de, sonradan kısaca göz attığım tarihi gerçeklerde meğer olayların büyük oranda bu şekilde gerçekleşmiş olduğunu okudum. Uzun yıllar akılda kalıcı bir film mi? Pek sanmıyorum. Çünkü nedense o Er Ryan'ı Kurtarmak gibi bir hava yok maalesef filmde. Başlangıç bölümlerini biraz daha kısaltabilirlerdi. Garfield'ın oyunculuğu başarılı ancak kendisine ödül getirmeye yete

In [43]:
# Zero-Shot Test ve değerlendirme
true_labels = sst2_test["label"]
pred_labels = bulk_predict(sst2_test["sentence"])

print(f"True Labels: {true_labels}")
print(f"Predicted Labels: {pred_labels}")

# Accuracy hesapla
acc = accuracy_score(true_labels, pred_labels)
print(f"Zero-Shot SST2 Accuracy (100 örnek): {acc:.2f}")


You seem to be using the pipelines sequentially on GPU. In order to maximize efficiency please use a dataset
Setting `pad_token_id` to `eos_token_id`:128009 for open-end generation.
Setting `pad_token_id` to `eos_token_id`:128009 for open-end generation.


True Labels: [0, 0]
Predicted Labels: [0, 0]
Zero-Shot SST2 Accuracy (100 örnek): 1.00


In [44]:
# 3-Shot Test ve değerlendirme
true_labels = sst2_test["label"]
pred_labels = bulk_predict_for_few_shot(sst2_test["sentence"],sst2_train, 3)

print(f"True Labels: {true_labels}")
print(f"Predicted Labels: {pred_labels}")

# Accuracy hesapla
acc = accuracy_score(true_labels, pred_labels)
print(f"3-Shot SST2 Accuracy (100 örnek): {acc:.2f}")


/usr/local/lib/python3.11/dist-packages/transformers/generation/configuration_utils.py:628: UserWarning: `do_sample` is set to `False`. However, `temperature` is set to `0.6` -- this flag is only used in sample-based generation modes. You should set `do_sample=True` or unset `temperature`.
  warnings.warn(
/usr/local/lib/python3.11/dist-packages/transformers/generation/configuration_utils.py:633: UserWarning: `do_sample` is set to `False`. However, `top_p` is set to `0.9` -- this flag is only used in sample-based generation modes. You should set `do_sample=True` or unset `top_p`.
  warnings.warn(
Setting `pad_token_id` to `eos_token_id`:128009 for open-end generation.
Setting `pad_token_id` to `eos_token_id`:128009 for open-end generation.


True Labels: [0, 0]
Predicted Labels: [0, 1]
3-Shot SST2 Accuracy (100 örnek): 0.50
